# Credit risk scorecards using linear and logistic regression models
**Sections summary**
1. Create and clean 2 data subsets based on 'Duration'
2. Create train and validation sets for each subset
3. Select 4 features including 1 continuous and 1 non-binary categorical
4. Build scorecards using linear and logistic regression
5. Evaluation metrics

In [1]:
# Libraries

# Core
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

# Model
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

**DATA IMPORT**

In [2]:
# Import data

url = "https://github.com/edwatson36/regression-ml/blob/main/data/raw/GermanCreditData.csv"
df = pd.read_csv(url)

HTTPError: HTTP Error 404: Not Found

In [ ]:
# Check import has worked (shape should = 1000, 22)

print(df.shape)
df.head(1)

# Section 1

Step 1: Check and clean data - converted 'X' values in 'qPurpose' column to 10 ('other' category). No other issues found.

Step 2: Split data based on nDuration.



In [ ]:
# Step 1: Check and clean data

# Check column types # all int64 except qPurpose=object
print("\nColumn Names and Data Types:")
print(df.dtypes)

# Find unique values in qPurpose
print(f"\nUnique values in qPurpose: {df['qPurpose'].unique()}")
print(f"Number of 'X' in qPurpose: {df['qPurpose'].eq('X').sum()}")

# Convert 'X' to 10 (='other')
df['qPurpose'] = df['qPurpose'].replace('X', 10).astype(int)
print(f"\nUnique values in qPurpose after conversion: {df['qPurpose'].unique()}") # check conversion
print(f"qPurpose column type: {df['qPurpose'].dtype}") # check column type

# Check for blanks # no nulls
print("\nColumns and the number of NaNs")
print(df.isnull().sum())

# Check for duplicates
print(f"\nDuplicate rows: {df.duplicated().sum()}")


In [ ]:
# Histogram of all columns - visual check for issues/imbalances
df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Step 2: Split data

# Create lists of categorical and numerical variables for ease of future reference
categorical_cols = [col for col in df.columns if col.startswith('q')]
numerical_cols = [col for col in df.columns if col.startswith('n')]

print(f"Categorical columns: {categorical_cols}")
print(f"Numerical columns: {numerical_cols}")

In [ ]:
# Create 2 subsets based on 'nDuration'
df1 = df[df['nDuration'] <= 12].copy()
df2 = df[df['nDuration'] > 12].copy()

# List of DataFrames to iterate through
dfs = [df1, df2]

print(f"df1 shape: {df1.shape}")
print(f"df2 shape: {df2.shape}")

# Section 2

Step 1: Visual inspection of each subset using histograms to identify class imblances. Identifies the need to stratify based on target variable (more Goods than Bads). No intuition about particularly important predictive variables therefore no other stratification applied.

Step 2: Drop 'Bad' column, keep 'Good' column as target variable (y). This means that our models we be predicting 'Good' as 1, the positive class.

Step 3: Create train-validation splits. 20% chosen to maximise the amount of train data available whilst keeping a representative portion of data back for validation. Note that we have small amounts of data, particularly after creating the subsets based on duration.

In [ ]:
# Histogram of all columns - visual check for issues/imbalances
df1.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Histogram of all columns - visual check for issues/imbalances
df2.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Select target variable column, drop redundant column

# Drop 'Bad' column
for d in dfs:
    d.drop(['Bad'], axis=1, inplace=True)

# Check
print(f"df1 shape: {df1.shape}")
print(f"df2 shape: {df2.shape}")

In [ ]:
# Stratified train-test split

# Train-test split
# stratified on y because we have fewer bads than goods in both df1 and df2

# df1
x1 = df1.drop(columns=['Good'])
y1 = df1['Good']

x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.2, random_state=42, stratify=y1)

# Check same proportions of 0s/1s in train and test
proportion_df1 = pd.DataFrame({'y_train1': y_train1.value_counts(normalize=True), 'y_test1': y_test1.value_counts(normalize=True)})
print(f"Proportions for df1:\n{proportion_df1}\n")

# df2
x2 = df2.drop(columns=['Good'])
y2 = df2['Good']

x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.2, random_state=42, stratify=y2)

# Check same proportions of 0s/1s in train and test
proportion_df2 = pd.DataFrame({'y_train2': y_train2.value_counts(normalize=True), 'y_test2': y_test2.value_counts(normalize=True)})
print(f"Proportions for df2:\n{proportion_df2}\n")

# check
print(f"x_train1 shape: {x_train1.shape}")
print(f"y_train1 shape: {y_train1.shape}")
print(f"x_test1 shape: {x_test1.shape}")
print(f"y_test1 shape: {y_test1.shape}")
print(f"x_train2 shape: {x_train2.shape}")
print(f"y_train2 shape: {y_train2.shape}")
print(f"x_test2 shape: {x_test2.shape}")
print(f"y_test2 shape: {y_test2.shape}")

# Section 3

*Feature selection using LASSO regression*

Step 1:
* Shortlist features with highest absolute coefficients at the optimal LASSO regression penalty for binning and Information Value analysis
* Seeking features that have a high absolute coefficient and Information Value of >0.1



In [ ]:
# Scale numerical variables

# Define MinMaxScaler for each subset
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()

# Scale only the numerical columns and replace in the original DataFrame
x_train1[numerical_cols] = scaler1.fit_transform(x_train1[numerical_cols])
x_train2[numerical_cols] = scaler2.fit_transform(x_train2[numerical_cols])

# Fit test sets to MinMaxScale
x_test1[numerical_cols] = scaler1.transform(x_test1[numerical_cols])
x_test2[numerical_cols] = scaler2.transform(x_test2[numerical_cols])


In [ ]:
# Categorical variables

# One-hot encode categorical variables
x_train1_enc = pd.get_dummies(x_train1, columns=categorical_cols, drop_first=True)
x_train2_enc = pd.get_dummies(x_train2, columns=categorical_cols, drop_first=True)

x_test1_enc = pd.get_dummies(x_test1, columns=categorical_cols, drop_first=True)
x_test1_enc = x_test1.reindex(columns=x_train1_enc.columns, fill_value=0) # ensure train and test have the same columns to prevent errors when testing the fitted model
x_test2_enc = pd.get_dummies(x_test2, columns=categorical_cols, drop_first=True)
x_test2_enc = x_test2.reindex(columns=x_train2_enc.columns, fill_value=0) # ensure train and test have the same columns to prevent errors when testing the fitted model

# Create list of names of encoded categorical columns
categorical_cols_enc1 = [col for col in x_train1_enc.columns if col.startswith('q')]
print(f"Categorical columns x_train1: {categorical_cols_enc1}")
categorical_cols_enc1_ts = [col for col in x_test1_enc.columns if col.startswith('q')]
print(f"Categorical columns x_test1: {categorical_cols_enc1_ts}")
categorical_cols_enc2 = [col for col in x_train2_enc.columns if col.startswith('q')]
print(f"Categorical columns x_train2: {categorical_cols_enc2}")
categorical_cols_enc2_ts = [col for col in x_test2_enc.columns if col.startswith('q')]
print(f"Categorical columns x_test2: {categorical_cols_enc2_ts}")

In [ ]:
# Use LASSO regression to determine which variables are useful

L1 = 0.1

# df1
reg = LassoCV()
reg.fit(x_train1_enc, y_train1)
coefs1 = pd.DataFrame({'feature': x_train1_enc.columns, 'coef1': reg.coef_})

# df2
reg.fit(x_train2_enc, y_train2)
coefs2 = pd.DataFrame({'feature': x_train2_enc.columns, 'coef2': reg.coef_})

# Merge coefficient results into one df
coef_df = pd.merge(coefs1, coefs2, on='feature')

# Sort
coef_df.sort_values(by=['coef1', 'coef2'], ascending=False, inplace=True)

In [ ]:
# Plot coefficients at optimal penalty
fig = go.Figure()

# Add coef1
fig.add_trace(go.Bar(x=coef_df['feature'],y=coef_df['coef1'], name='Coef 1',marker_color='blue'))

# Add coef2
fig.add_trace(go.Bar(x=coef_df['feature'],y=coef_df['coef2'],name='Coef 2',marker_color='orange'))

# Update layout
fig.update_layout(
    title='Comparison of Coefficients',
    xaxis_title='Feature', xaxis=dict(tickangle=45, tickmode='array'),
    yaxis_title='Coefficient Value',
    barmode='group', template='plotly_white', width=1200, height=500
)

fig.show()

*Shortlist of variables for binning using coefficient threshold of >=abs(0.1)*

* Numerical variables:
    * x1 - Age, Duration
    * x2 - Duration, Installp
* Categorical variables:
    * x1 - Checking_4, Coapp_3, Property_4
    * x2 - Checking_4, Checking_3, Purpose_1, Coapp_3

* Best 4 variables (in order):
    * x1 - qChecking, qProperty, nDuration, qCoapp
    * x2 - qChecking, nDuration, qPurpose, qCoapp



*Binning numerical variables*

In [ ]:
# Numerical variables binning

# Histogram of each shortlisted variable to identify potential bin thresholds

# Reverse MinMax Scaling for interpretability of bin thresholds
x_train1_fs = pd.DataFrame(scaler1.inverse_transform(x_train1[numerical_cols]), columns=x_train1[numerical_cols].columns, index=x_train1.index)
x_train2_fs = pd.DataFrame(scaler2.inverse_transform(x_train2[numerical_cols]), columns=x_train2[numerical_cols].columns, index=x_train2.index)

# Split data into Good and Bad for x_train1
good_credit1 = x_train1_fs[y_train1 == 1]
bad_credit1 = x_train1_fs[y_train1 == 0]

# Split data into Good and Bad for x_train2
good_credit2 = x_train2_fs[y_train2 == 1]
bad_credit2 = x_train2_fs[y_train2 == 0]

# Add goods for % calc
x_train1_fs['Good'] = y_train1.values
x_train2_fs['Good'] = y_train2.values

# Plot histograms
plt.figure(figsize=(15, 10))

# Create 3x2 subplots
fig = make_subplots(rows=3, cols=2, subplot_titles=[
    'nDuration Distribution (x_train1)',
    'nAge Distribution (x_train1)',
    'nDuration Distribution (x_train2)',
    'nInstallp Distribution (x_train2)',
    'nAmount Distribution (xtrain_2)',
    'nInstallp Distribution (x_train1)'
])

# Plot 1: nDuration (x_train1)
fig.add_trace(go.Histogram(x=good_credit1['nDuration'], name='Good Credit', marker_color='blue', opacity=0.5, xbins=dict(start=0, end=good_credit1['nDuration'].max()+1, size=1)), row=1, col=1)
fig.add_trace(go.Histogram(x=bad_credit1['nDuration'], name='Bad Credit', marker_color='red', opacity=0.5, xbins=dict(start=0, end=bad_credit1['nDuration'].max()+1, size=1)), row=1, col=1)
# Group by nDuration and calculate % of good applicants
dur1_good_pct = x_train1_fs.groupby('nDuration')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=dur1_good_pct.index,y=dur1_good_pct.values,mode='lines+markers',name='% of Good Applicants', line=dict(color='blue'), marker=dict(size=6, color='blue')), row=1, col=1)

# Plot 2: nAge (x_train1)
fig.add_trace(go.Histogram(x=good_credit1['nAge'], name='Good Credit', marker_color='blue', opacity=0.5, xbins=dict(start=18, end=good_credit1['nAge'].max()+1, size=5)), row=1, col=2)
fig.add_trace(go.Histogram(x=bad_credit1['nAge'], name='Bad Credit', marker_color='red', opacity=0.5, xbins=dict(start=18, end=bad_credit1['nAge'].max()+1, size=5)), row=1, col=2)
# Group by nAge and calculate % of good applicants
age_good_pct = x_train1_fs.groupby('nAge')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=age_good_pct.index,y=age_good_pct.values,mode='lines+markers',name='% of Good Applicants', line=dict(color='blue'), marker=dict(size=6, color='blue')), row=1, col=2)

# Plot 3: nDuration (x_train2)
fig.add_trace(go.Histogram(x=good_credit2['nDuration'], name='Good Credit', marker_color='blue', opacity=0.5, xbins=dict(start=10, end=good_credit2['nDuration'].max()+1, size=1)), row=2, col=1)
fig.add_trace(go.Histogram(x=bad_credit2['nDuration'], name='Bad Credit', marker_color='red', opacity=0.5, xbins=dict(start=10, end=bad_credit2['nDuration'].max()+1, size=1)), row=2, col=1)
# Group by nDuration and calculate % of good applicants
dur2_good_pct = x_train2_fs.groupby('nDuration')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=dur2_good_pct.index,y=dur2_good_pct.values,mode='lines+markers',name='% of Good Applicants', line=dict(color='blue'), marker=dict(size=6, color='blue')), row=2, col=1)

# Plot 4: nInstallp (x_train2)
fig.add_trace(go.Histogram(x=good_credit2['nInstallp'], name='Good Credit', marker_color='blue', opacity=0.5, xbins=dict(start=0, end=good_credit2['nInstallp'].max()+1, size=1)), row=2, col=2)
fig.add_trace(go.Histogram(x=bad_credit2['nInstallp'], name='Bad Credit', marker_color='red', opacity=0.5, xbins=dict(start=0, end=bad_credit2['nInstallp'].max()+1, size=1)), row=2, col=2)
# Group by nInstallp and calculate % of good applicants
inst_good_pct = x_train2_fs.groupby('nInstallp')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=inst_good_pct.index,y=inst_good_pct.values,mode='lines+markers',name='% of Good Applicants', line=dict(color='blue'), marker=dict(size=6, color='blue')), row=2, col=2)

# Plot 5: nAmount (x_train2)
fig.add_trace(go.Histogram(x=good_credit2['nAmount'], name='Good Credit', marker_color='blue', opacity=0.5, xbins=dict(start=0, end=good_credit2['nAmount'].max()+1, size=2500)), row=3, col=1)
fig.add_trace(go.Histogram(x=bad_credit2['nAmount'], name='Bad Credit', marker_color='red', opacity=0.5, xbins=dict(start=0, end=bad_credit2['nAmount'].max()+1, size=2500)), row=3, col=1)
# Group by nInstallp and calculate % of good applicants
#inst_good_pct = x_train2_fs.groupby('nAmount')['Good'].mean() * 100
#fig.add_trace(go.Scatter(x=inst_good_pct.index,y=inst_good_pct.values,mode='lines+markers',name='% of Good Applicants', line=dict(color='blue'), marker=dict(size=6, color='blue')), row=3, col=1)

# Plot 5: nInstallp (x_train1)
fig.add_trace(go.Histogram(x=good_credit1['nInstallp'], name='Good Credit', marker_color='blue', opacity=0.5, xbins=dict(start=0, end=good_credit1['nInstallp'].max()+1, size=1)), row=3, col=2)
fig.add_trace(go.Histogram(x=bad_credit1['nInstallp'], name='Bad Credit', marker_color='red', opacity=0.5, xbins=dict(start=0, end=bad_credit1['nInstallp'].max()+1, size=1)), row=3, col=2)

# Group by nInstallp and calculate % of good applicants
inst_good_pct = x_train1_fs.groupby('nInstallp')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=inst_good_pct.index, y=inst_good_pct.values, mode='lines+markers', name='% of Good Applicants', line=dict(color='blue'), marker=dict(size=6, color='blue')), row=3, col=2)



# Update layout
fig.update_layout(
    title='Numerical variables - good/bad distributions',
    barmode='overlay',
    width=900,
    height=700,
    template='plotly_white',
    showlegend=True
)

fig.show()

# x1 Duration - 5 bins 4-5, 6-7, 8-9, 10-11, 12; 3 bins 4-7, 8-11, 12
# x1 Age - decades - 5 bins 19-29, 30-39, 40-49, 50-59, 60+ ;
# x2 Duration - longer is higher risk - 4 c12 month bins 13-24, 25-36, 37-48, 49+ or 13-23, 24-35, 36-47, 48+
# x2 Installp - 1+2 vs. 3+4, 1+2+3 vs. 4
# x2 nAmount - 0-5k, 5-10k, >10k


In [ ]:
# Work out the best binning option for each variable and calculate the WOE and IV of each bin

In [ ]:
# 3 binning options for x_train1['nDuration']

# Decision - x1_dur_o2 - 3 buckets - good IV of 0.3 and 5 buckets IV of 0.843 is possibly over-predicting

ep = 1e-6

#################################
# x1_dur_o1
#################################
bins = [0, 11, 12]  # Define the bin edges
labels = ['4-11 mnth', '12 mnth']  # Labels for the bins

# Bin the 'nDuration' values
x_train1_fs['nDuration_bin'] = pd.cut(x_train1_fs['nDuration'], bins=bins, labels=labels, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_dur_o1 = pd.crosstab(x_train1_fs['nDuration_bin'], x_train1_fs['Good'], dropna=False).reset_index()
x1_dur_o1.columns = ['Bin', 'Bads', 'Goods']
x1_dur_o1 = x1_dur_o1[['Bin','Goods','Bads', ]] # re-order columns

# Calculate G_i/G and B_i/B
x1_dur_o1['Good%'] = x1_dur_o1['Goods'] / x1_dur_o1['Goods'].sum()
x1_dur_o1['Bad%'] = x1_dur_o1['Bads'] / x1_dur_o1['Bads'].sum()

# Calculate WOE for each row
x1_dur_o1['WOE'] = np.log(x1_dur_o1['Good%'] / x1_dur_o1['Bad%'])

# Calculate IV for each row
x1_dur_o1['IV'] = (x1_dur_o1['Good%'] - x1_dur_o1['Bad%']) * x1_dur_o1['WOE']

# Calculate total IV
total_IV = x1_dur_o1['IV'].sum()

# Print results
print(x1_dur_o1)
print(f"Total IV: {total_IV}")

#################################
# x1_dur_o2
#################################
bins_x1dur2 = [0, 7, 11, 12]  # Define the bin edges
labels_x1dur2 = ['4-7','8-11', '12']  # Labels for the bins

# Bin the 'nDuration' values
x_train1_fs['nDuration_bin'] = pd.cut(x_train1_fs['nDuration'], bins=bins_x1dur2, labels=labels_x1dur2, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_dur_o2 = pd.crosstab(x_train1_fs['nDuration_bin'], x_train1_fs['Good'], dropna=False).reset_index()
x1_dur_o2.columns = ['Bin', 'Bads', 'Goods']
x1_dur_o2 = x1_dur_o2[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_dur_o2['Good%'] = x1_dur_o2['Goods'] / x1_dur_o2['Goods'].sum()
x1_dur_o2['Bad%'] = x1_dur_o2['Bads'] / x1_dur_o2['Bads'].sum()

# Calculate WOE for each row
x1_dur_o2['WOE'] = np.log(x1_dur_o2['Good%'] / x1_dur_o2['Bad%'])

# Calculate IV for each row
x1_dur_o2['IV'] = (x1_dur_o2['Good%'] - x1_dur_o2['Bad%']) * x1_dur_o2['WOE']

# Calculate total IV
total_IV = x1_dur_o2['IV'].sum()

# Print results
print(x1_dur_o2)
print(f"Total IV: {total_IV}")

#################################
# x1_dur_o3
#################################
bins = [0, 5, 7, 9, 11, 12]  # Define the bin edges
labels = ['4-5 mnth','6-7 mnth', '8-9 mnth', '10-11 mnth', '12 mnth']  # Labels for the bins

# Bin the 'nDuration' values
x_train1_fs['nDuration_bin'] = pd.cut(x_train1_fs['nDuration'], bins=bins, labels=labels, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_dur_o3 = pd.crosstab(x_train1_fs['nDuration_bin'], x_train1_fs['Good'], dropna=False).reset_index()
x1_dur_o3.columns = ['Bin', 'Bads', 'Goods']
x1_dur_o3 = x1_dur_o3[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_dur_o3['Good%'] = (x1_dur_o3['Goods']+ep) / x1_dur_o3['Goods'].sum()
x1_dur_o3['Bad%'] = (x1_dur_o3['Bads']+ep) / x1_dur_o3['Bads'].sum()

# Calculate WOE for each row
x1_dur_o3['WOE'] = np.log(x1_dur_o3['Good%'] / x1_dur_o3['Bad%'])

# Calculate IV for each row
x1_dur_o3['IV'] = (x1_dur_o3['Good%'] - x1_dur_o3['Bad%']) * x1_dur_o3['WOE']

# Calculate total IV
total_IV = x1_dur_o3['IV'].sum()

# Print results
print(x1_dur_o3)
print(f"Total IV: {total_IV}")


In [ ]:
# binning of x_train1['nAge']

# Decision - only include if categorical variables have less good IVs than 0.21

bins = [0, 30, 40, 50, 60, np.inf]
labels = ['19-29', '30-39', '40-49', '50-59', '60+']

# Bin the 'nAge' values
x_train1_fs['nAge_bin'] = pd.cut(x_train1_fs['nAge'], bins=bins, labels=labels, right=False)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_age = pd.crosstab(x_train1_fs['nAge_bin'], x_train1_fs['Good'], dropna=False).reset_index()
x1_age.columns = ['Bin', 'Bads', 'Goods']
x1_age = x1_age[['Bin', 'Goods', 'Bads']]  # re-order columns

# Calculate G_i/G and B_i/B
x1_age['Good%'] = (x1_age['Goods'] + ep) / (x1_age['Goods'].sum() + ep)
x1_age['Bad%'] = (x1_age['Bads'] + ep) / (x1_age['Bads'].sum() + ep)

# Calculate WOE for each row
x1_age['WOE'] = np.log(x1_age['Good%'] / x1_age['Bad%'])

# Calculate IV for each row
x1_age['IV'] = (x1_age['Good%'] - x1_age['Bad%']) * x1_age['WOE']

# Calculate total IV
total_IV = x1_age['IV'].sum()

# Print results
print(x1_age)
print(f"Total IV: {total_IV}")


In [ ]:
# Interestingly, IV of Installp for subset 1 is much stronger despite LASSO regression results - 0.22. This is a better IV than age but with fewer buckets.

# Decision - potentially use option 3 of this feature if other categorical variables have lower IVs.

#################################
# x1_installp_o1
#################################
# Option 1: Bin edges [0, 2, np.inf]
bins_o1 = [0, 2, np.inf]
labels_o1 = ['0-2 Installp', '3+ Installp']

# Bin the 'nInstallp' values
x_train1_fs['nInstallp_bin'] = pd.cut(x_train1_fs['nInstallp'], bins=bins_o1, labels=labels_o1, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_installp_o1 = pd.crosstab(x_train1_fs['nInstallp_bin'], x_train1_fs['Good'], dropna=False).reset_index()
x1_installp_o1.columns = ['Bin', 'Bads', 'Goods']
x1_installp_o1 = x1_installp_o1[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_installp_o1['Good%'] = x1_installp_o1['Goods'] / x1_installp_o1['Goods'].sum()
x1_installp_o1['Bad%'] = x1_installp_o1['Bads'] / x1_installp_o1['Bads'].sum()

# Calculate WOE for each row
x1_installp_o1['WOE'] = np.log(x1_installp_o1['Good%'] / x1_installp_o1['Bad%'])

# Calculate IV for each row
x1_installp_o1['IV'] = (x1_installp_o1['Good%'] - x1_installp_o1['Bad%']) * x1_installp_o1['WOE']

# Calculate total IV
total_IV_o1 = x1_installp_o1['IV'].sum()

# Print results
print(x1_installp_o1)
print(f"Total IV for x1_installp_o1: {total_IV_o1}")


#################################
# x1_installp_o2
#################################
# Option 2: Bin edges [0, 3, np.inf]
bins_o2 = [0, 3, np.inf]
labels_o2 = ['0-3 Installp', '4+ Installp']

# Bin the 'nInstallp' values
x_train1_fs['nInstallp_bin'] = pd.cut(x_train1_fs['nInstallp'], bins=bins_o2, labels=labels_o2, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_installp_o2 = pd.crosstab(x_train1_fs['nInstallp_bin'], x_train1_fs['Good'], dropna=False).reset_index()
x1_installp_o2.columns = ['Bin', 'Bads', 'Goods']
x1_installp_o2 = x1_installp_o2[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_installp_o2['Good%'] = x1_installp_o2['Goods'] / x1_installp_o2['Goods'].sum()
x1_installp_o2['Bad%'] = x1_installp_o2['Bads'] / x1_installp_o2['Bads'].sum()

# Calculate WOE for each row
x1_installp_o2['WOE'] = np.log(x1_installp_o2['Good%'] / x1_installp_o2['Bad%'])

# Calculate IV for each row
x1_installp_o2['IV'] = (x1_installp_o2['Good%'] - x1_installp_o2['Bad%']) * x1_installp_o2['WOE']

# Calculate total IV
total_IV_o2 = x1_installp_o2['IV'].sum()

# Print results
print(x1_installp_o2)
print(f"Total IV for x1_installp_o2: {total_IV_o2}")


#################################
# x1_installp_o3
#################################
# Option 3: Bin edges [0, 2, 3, np.inf]
bins_o3 = [0, 2, 3, np.inf]
labels_o3 = ['0-2', '3', '4+']

# Bin the 'nInstallp' values
x_train1_fs['nInstallp_bin'] = pd.cut(x_train1_fs['nInstallp'], bins=bins_o3, labels=labels_o3, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_installp_o3 = pd.crosstab(x_train1_fs['nInstallp_bin'], x_train1_fs['Good'], dropna=False).reset_index()
x1_installp_o3.columns = ['Bin', 'Bads', 'Goods']
x1_installp_o3 = x1_installp_o3[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_installp_o3['Good%'] = x1_installp_o3['Goods'] / x1_installp_o3['Goods'].sum()
x1_installp_o3['Bad%'] = x1_installp_o3['Bads'] / x1_installp_o3['Bads'].sum()

# Calculate WOE for each row
x1_installp_o3['WOE'] = np.log(x1_installp_o3['Good%'] / x1_installp_o3['Bad%'])

# Calculate IV for each row
x1_installp_o3['IV'] = (x1_installp_o3['Good%'] - x1_installp_o3['Bad%']) * x1_installp_o3['WOE']

# Calculate total IV
total_IV_o3 = x1_installp_o3['IV'].sum()

# Print results
print(x1_installp_o3)
print(f"Total IV for x1_installp_o3: {total_IV_o3}")


In [ ]:
# binning of x_train2['nDuration']

# 2 binning options considered

# Decision - option 2 - better IV at 0.12

#################################
# x2_dur_o1
#################################
bins = [0, 24, 36, 48, np.inf]
labels = ['13-24 mnth', '25-36 mnth', '37-48 mnth', '49+ mnth']

# Bin the 'nDuration' values
x_train2_fs['nDuration_bin'] = pd.cut(x_train2_fs['nDuration'], bins=bins, labels=labels, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_dur_o1 = pd.crosstab(x_train2_fs['nDuration_bin'], x_train2_fs['Good'], dropna=False).reset_index()
x2_dur_o1.columns = ['Bin', 'Bads', 'Goods']
x2_dur_o1 = x2_dur_o1[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_dur_o1['Good%'] = x2_dur_o1['Goods'] / x2_dur_o1['Goods'].sum()
x2_dur_o1['Bad%'] = x2_dur_o1['Bads'] / x2_dur_o1['Bads'].sum()

# Calculate WOE for each row
x2_dur_o1['WOE'] = np.log(x2_dur_o1['Good%'] / x2_dur_o1['Bad%'])

# Calculate IV for each row
x2_dur_o1['IV'] = (x2_dur_o1['Good%'] - x2_dur_o1['Bad%']) * x2_dur_o1['WOE']

# Calculate total IV
total_IV_o1 = x2_dur_o1['IV'].sum()

# Print results
print(x2_dur_o1)
print(f"Total IV for x2_dur_o1: {total_IV_o1}")

#################################
# x2_dur_o2
#################################
bins_dur2 = [0, 23, 35, 47, np.inf]
labels_dur2 = ['13-23', '24-35', '36-47', '48+']

# Bin the 'nDuration' values
x_train2_fs['nDuration_bin'] = pd.cut(x_train2_fs['nDuration'], bins=bins_dur2, labels=labels_dur2, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_dur_o2 = pd.crosstab(x_train2_fs['nDuration_bin'], x_train2_fs['Good'], dropna=False).reset_index()
x2_dur_o2.columns = ['Bin', 'Bads', 'Goods']
x2_dur_o2 = x2_dur_o2[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_dur_o2['Good%'] = x2_dur_o2['Goods'] / x2_dur_o2['Goods'].sum()
x2_dur_o2['Bad%'] = x2_dur_o2['Bads'] / x2_dur_o2['Bads'].sum()

# Calculate WOE for each row
x2_dur_o2['WOE'] = np.log(x2_dur_o2['Good%'] / x2_dur_o2['Bad%'])

# Calculate IV for each row
x2_dur_o2['IV'] = (x2_dur_o2['Good%'] - x2_dur_o2['Bad%']) * x2_dur_o2['WOE']

# Calculate total IV
total_IV_o2 = x2_dur_o2['IV'].sum()

# Print results
print(x2_dur_o2)
print(f"Total IV for x2_dur_o2: {total_IV_o2}")


In [ ]:
# x2
# Installp

# 3 binning options considered

# Decision - don't use this feature: as suspected from histogram, very low information value

#################################
# x2_installp_o1
#################################
# Option 1: Bin edges [0, 2, np.inf]
bins_o1 = [0, 2, np.inf]
labels_o1 = ['0-2 Installp', '3+ Installp']

# Bin the 'nInstallp' values
x_train2_fs['nInstallp_bin'] = pd.cut(x_train2_fs['nInstallp'], bins=bins_o1, labels=labels_o1, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_installp_o1 = pd.crosstab(x_train2_fs['nInstallp_bin'], x_train2_fs['Good'], dropna=False).reset_index()
x2_installp_o1.columns = ['Bin', 'Bads', 'Goods']
x2_installp_o1 = x2_installp_o1[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_installp_o1['Good%'] = x2_installp_o1['Goods'] / x2_installp_o1['Goods'].sum()
x2_installp_o1['Bad%'] = x2_installp_o1['Bads'] / x2_installp_o1['Bads'].sum()

# Calculate WOE for each row
x2_installp_o1['WOE'] = np.log(x2_installp_o1['Good%'] / x2_installp_o1['Bad%'])

# Calculate IV for each row
x2_installp_o1['IV'] = (x2_installp_o1['Good%'] - x2_installp_o1['Bad%']) * x2_installp_o1['WOE']

# Calculate total IV
total_IV_o1 = x2_installp_o1['IV'].sum()

# Print results
print(x2_installp_o1)
print(f"Total IV for x2_installp_o1: {total_IV_o1}")


#################################
# x2_installp_o2
#################################
# Option 2: Bin edges [0, 3, np.inf]
bins_o2 = [0, 3, np.inf]
labels_o2 = ['0-3 Installp', '4+ Installp']

# Bin the 'nInstallp' values
x_train2_fs['nInstallp_bin'] = pd.cut(x_train2_fs['nInstallp'], bins=bins_o2, labels=labels_o2, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_installp_o2 = pd.crosstab(x_train2_fs['nInstallp_bin'], x_train2_fs['Good'], dropna=False).reset_index()
x2_installp_o2.columns = ['Bin', 'Bads', 'Goods']
x2_installp_o2 = x2_installp_o2[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_installp_o2['Good%'] = x2_installp_o2['Goods'] / x2_installp_o2['Goods'].sum()
x2_installp_o2['Bad%'] = x2_installp_o2['Bads'] / x2_installp_o2['Bads'].sum()

# Calculate WOE for each row
x2_installp_o2['WOE'] = np.log(x2_installp_o2['Good%'] / x2_installp_o2['Bad%'])

# Calculate IV for each row
x2_installp_o2['IV'] = (x2_installp_o2['Good%'] - x2_installp_o2['Bad%']) * x2_installp_o2['WOE']

# Calculate total IV
total_IV_o2 = x2_installp_o2['IV'].sum()

# Print results
print(x2_installp_o2)
print(f"Total IV for x2_installp_o2: {total_IV_o2}")


#################################
# x2_installp_o3
#################################
# Option 3: Bin edges [0, 2, 3, np.inf]
bins_o3 = [0, 2, 3, np.inf]
labels_o3 = ['0-2 Installp', '3 Installp', '4+ Installp']

# Bin the 'nInstallp' values
x_train2_fs['nInstallp_bin'] = pd.cut(x_train2_fs['nInstallp'], bins=bins_o3, labels=labels_o3, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_installp_o3 = pd.crosstab(x_train2_fs['nInstallp_bin'], x_train2_fs['Good'], dropna=False).reset_index()
x2_installp_o3.columns = ['Bin', 'Bads', 'Goods']
x2_installp_o3 = x2_installp_o3[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_installp_o3['Good%'] = x2_installp_o3['Goods'] / x2_installp_o3['Goods'].sum()
x2_installp_o3['Bad%'] = x2_installp_o3['Bads'] / x2_installp_o3['Bads'].sum()

# Calculate WOE for each row
x2_installp_o3['WOE'] = np.log(x2_installp_o3['Good%'] / x2_installp_o3['Bad%'])

# Calculate IV for each row
x2_installp_o3['IV'] = (x2_installp_o3['Good%'] - x2_installp_o3['Bad%']) * x2_installp_o3['WOE']

# Calculate total IV
total_IV_o3 = x2_installp_o3['IV'].sum()

# Print results
print(x2_installp_o3)
print(f"Total IV for x2_installp_o3: {total_IV_o3}")


In [ ]:
# x2
# nAmount

# 2 binning options considered
# Decision - don't use this feature - IV <0.1


# Option 1: 0-5k, 5-10k, >10k - IV = 0.05
bins_am = [0, 5000, 10000, np.inf]
labels_am = ['<=5k', '5-10k', '>10k']

# Bin the 'nAmount' values
x_train2_fs['nAmount_bin'] = pd.cut(x_train2_fs['nAmount'], bins=bins_am, labels=labels_am, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_amount_o1 = pd.crosstab(x_train2_fs['nAmount_bin'], x_train2_fs['Good'], dropna=False).reset_index()
x2_amount_o1.columns = ['Bin', 'Bads', 'Goods']
x2_amount_o1 = x2_amount_o1[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_amount_o1['Good%'] = x2_amount_o1['Goods'] / x2_amount_o1['Goods'].sum()
x2_amount_o1['Bad%'] = x2_amount_o1['Bads'] / x2_amount_o1['Bads'].sum()

# Calculate WOE for each row
x2_amount_o1['WOE'] = np.log(x2_amount_o1['Good%'] / x2_amount_o1['Bad%'])

# Calculate IV for each row
x2_amount_o1['IV'] = (x2_amount_o1['Good%'] - x2_amount_o1['Bad%']) * x2_amount_o1['WOE']

# Calculate total IV
total_IV_o1 = x2_amount_o1['IV'].sum()

# Print results
print(x2_amount_o1)
print(f"Total IV for x2_amount_o1: {total_IV_o1}")


# Option 2: 0-2.5k, 2.5-5k 5-10k, >10k - IV = 0.08
bins_am2 = [0, 2500, 5000, 10000, np.inf]
labels_am2 = ['<=2.5k', '2.5-5k', '5-10k', '>10k']

# Bin the 'nAmount' values
x_train2_fs['nAmount_bin'] = pd.cut(x_train2_fs['nAmount'], bins=bins_am2, labels=labels_am2, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_amount_o2 = pd.crosstab(x_train2_fs['nAmount_bin'], x_train2_fs['Good'], dropna=False).reset_index()
x2_amount_o2.columns = ['Bin', 'Bads', 'Goods']
x2_amount_o2 = x2_amount_o2[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_amount_o2['Good%'] = x2_amount_o2['Goods'] / x2_amount_o2['Goods'].sum()
x2_amount_o2['Bad%'] = x2_amount_o2['Bads'] / x2_amount_o2['Bads'].sum()

# Calculate WOE for each row
x2_amount_o2['WOE'] = np.log(x2_amount_o2['Good%'] / x2_amount_o2['Bad%'])

# Calculate IV for each row
x2_amount_o2['IV'] = (x2_amount_o2['Good%'] - x2_amount_o2['Bad%']) * x2_amount_o2['WOE']

# Calculate total IV
total_IV_o2 = x2_amount_o2['IV'].sum()

# Print results
print(x2_amount_o2)
print(f"Total IV for x2_amount_o2: {total_IV_o2}")

*Summary of numerical variables*

* Subset 1:
    * Use nDuration in 3 buckets - IV = 0.3
    * Potentially use nInstallp in 3 buckets too - IV = 0.22
* Subset 2:
    * Use nDuration in 4 buckets - IV = 0.12

*Binning categorical variables*

* x1 - Checking_4, Coapp_3, Property_4
* x2 - Checking_4, Checking_3, Purpose_1, Coapp_3

In [ ]:
# Visualise volumes and % Good in each shortlisted feature

# Add goods for % calc
x_train1['Good'] = y_train1.values
x_train2['Good'] = y_train2.values

# Create a bar plot for 'qChecking'
qChecking_counts = x_train1['qChecking'].value_counts().reset_index()
qChecking_counts.columns = ['Category', 'Count']

# Create a bar plot for 'qCoapp'
qCoapp_counts = x_train1['qCoapp'].value_counts().reset_index()
qCoapp_counts.columns = ['Category', 'Count']

# Create a bar plot for 'qProperty'
qProperty_counts = x_train1['qProperty'].value_counts().reset_index()
qProperty_counts.columns = ['Category', 'Count']

# Create the bar graph for all three variables
fig = go.Figure()

# Add bar for qChecking
fig.add_trace(go.Bar(x=qChecking_counts['Category'],y=qChecking_counts['Count'],name='qChecking',marker=dict(color='blue')
))
chk_good_pct = x_train1.groupby('qChecking')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=chk_good_pct.index,y=chk_good_pct.values,mode='lines+markers',name='% of Good Checking', line=dict(color='blue'), marker=dict(size=6, color='blue')))

# Add bar for qCoapp
fig.add_trace(go.Bar(x=qCoapp_counts['Category'],y=qCoapp_counts['Count'],name='qCoapp',marker=dict(color='green')
))
coap_good_pct = x_train1.groupby('qCoapp')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=coap_good_pct.index,y=coap_good_pct.values,mode='lines+markers',name='% of Good Coap', line=dict(color='green'), marker=dict(size=6, color='green')))

# Add bar for qProperty
fig.add_trace(go.Bar(x=qProperty_counts['Category'],y=qProperty_counts['Count'],name='qProperty',marker=dict(color='red')
))
prop_good_pct = x_train1.groupby('qProperty')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=prop_good_pct.index,y=prop_good_pct.values,mode='lines+markers',name='% of Good Coap', line=dict(color='red'), marker=dict(size=6, color='red')))

# Update layout
fig.update_layout(
    title='Volume of Applicants and % of Goods by Category - subset 1 only',
    barmode='group',
    xaxis_title='Category',
    yaxis_title='Volume of Applicants',
    template='plotly_white'  # You can change the template for the look of the plot
)

# Show the plot
fig.show()

# Checking - keep as 4 buckets (clear difference in % goods in each bucket and logical difference between buckets)
# Coap - 1+2 vs. 3 (logical grouping of no guarentor vs. guarentor reflected in % of goods)
# Property 1, 2+3, 4 (real estate, other collateral, unknown - logical grouping and reflected in % of goods)

In [ ]:
# x1 Checking
#################################

# Keeping original categories
# IV = 0.65, but this is expected for this variable

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_checking = pd.crosstab(x_train1['qChecking'], x_train1['Good'], dropna=False).reset_index()
x1_checking.columns = ['Bin', 'Bads', 'Goods']
x1_checking = x1_checking[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_checking['Good%'] = x1_checking['Goods'] / x1_checking['Goods'].sum()
x1_checking['Bad%'] = x1_checking['Bads'] / x1_checking['Bads'].sum()

# Calculate WOE for each row
x1_checking['WOE'] = np.log(x1_checking['Good%'] / x1_checking['Bad%'])

# Calculate IV for each row
x1_checking['IV'] = (x1_checking['Good%'] - x1_checking['Bad%']) * x1_checking['WOE']

# Calculate total IV
total_IV_chk = x1_checking['IV'].sum()

# Print results
print(x1_checking)
print(f"Total IV for x1_checking: {total_IV_chk}")

In [ ]:
# x1 Coapp

# 2 bins - very high IV 0.87 but expected

# Bins
bins_coapp1 = [0, 2, np.inf]
labels_coapp1 = ['No Gurantor', 'Guarantor']

# Bin the 'nCoapp' values
x_train1['qCoapp_bin'] = pd.cut(x_train1['qCoapp'], bins=bins_coapp1, labels=labels_coapp1, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_coap = pd.crosstab(x_train1['qCoapp_bin'], x_train1['Good'], dropna=False).reset_index()
x1_coap.columns = ['Bin', 'Bads', 'Goods']
x1_coap = x1_coap[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_coap['Good%'] = x1_coap['Goods'] / x1_coap['Goods'].sum()
x1_coap['Bad%'] = (x1_coap['Bads']+ep) / x1_coap['Bads'].sum()

# Calculate WOE for each row
x1_coap['WOE'] = np.log(x1_coap['Good%'] / x1_coap['Bad%'])

# Calculate IV for each row
x1_coap['IV'] = (x1_coap['Good%'] - x1_coap['Bad%']) * x1_coap['WOE']

# Calculate total IV
total_IV_coap = x1_coap['IV'].sum()

# Print results
print(x1_coap)
print(f"Total IV for x1_coap: {total_IV_coap}")

In [ ]:
# x1 Property

# Good IV of 0.28

# Bins
bins_prop1 = [0, 1, 3, np.inf]
labels_prop1 = ['Real_estate', 'Other_collat', 'Unk_Na']

# Bin the 'qProperty' values
x_train1['qProp_bin'] = pd.cut(x_train1['qProperty'], bins=bins_prop1, labels=labels_prop1, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x1_prop = pd.crosstab(x_train1['qProp_bin'], x_train1['Good'], dropna=False).reset_index()
x1_prop.columns = ['Bin', 'Bads', 'Goods']
x1_prop = x1_prop[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x1_prop['Good%'] = x1_prop['Goods'] / x1_prop['Goods'].sum()
x1_prop['Bad%'] = (x1_prop['Bads'] + ep) / x1_prop['Bads'].sum()

# Calculate WOE for each row
x1_prop['WOE'] = np.log(x1_prop['Good%'] / x1_prop['Bad%'])

# Calculate IV for each row
x1_prop['IV'] = (x1_prop['Good%'] - x1_prop['Bad%']) * x1_prop['WOE']

# Calculate total IV
total_IV_prop = x1_prop['IV'].sum()

# Print results
print(x1_prop)
print(f"Total IV for x1_prop: {total_IV_prop}")

*x2*

In [ ]:
# Visualise volumes and % Good in each shortlisted feature

# Add goods for % calc
x_train1['Good'] = y_train1.values
x_train2['Good'] = y_train2.values

# Create a bar plot for 'qChecking'
qChecking_counts = x_train2['qChecking'].value_counts().reset_index()
qChecking_counts.columns = ['Category', 'Count']

# Create a bar plot for 'qCoapp'
qCoapp_counts = x_train2['qCoapp'].value_counts().reset_index()
qCoapp_counts.columns = ['Category', 'Count']

# Create a bar plot for 'qPurpose'
qPurpose_counts = x_train2['qPurpose'].value_counts().reset_index()
qPurpose_counts.columns = ['Category', 'Count']

# Create a bar plot for 'qEmployed'
qEmployed_counts = x_train2['qEmployed'].value_counts().reset_index()
qEmployed_counts.columns = ['Category', 'Count']

# Create a bar plot for 'qSavings'
qSavings_counts = x_train2['qSavings'].value_counts().reset_index()
qSavings_counts.columns = ['Category', 'Count']

# Create the bar graph for all three variables
fig = go.Figure()

# Add bar for qChecking
fig.add_trace(go.Bar(x=qChecking_counts['Category'], y=qChecking_counts['Count'], name='qChecking', marker=dict(color='blue')))
chk_good_pct = x_train2.groupby('qChecking')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=chk_good_pct.index, y=chk_good_pct.values, mode='lines+markers', name='% of Good Checking', line=dict(color='blue'), marker=dict(size=6, color='blue')))

# Add bar for qCoapp
fig.add_trace(go.Bar(x=qCoapp_counts['Category'], y=qCoapp_counts['Count'], name='qCoapp', marker=dict(color='green')))
coap_good_pct = x_train2.groupby('qCoapp')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=coap_good_pct.index, y=coap_good_pct.values, mode='lines+markers', name='% of Good Coap', line=dict(color='green'), marker=dict(size=6, color='green')))

# Add bar for qPurpose
fig.add_trace(go.Bar(x=qPurpose_counts['Category'], y=qPurpose_counts['Count'], name='qPurpose', marker=dict(color='red')))
purp_good_pct = x_train2.groupby('qPurpose')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=purp_good_pct.index, y=purp_good_pct.values, mode='lines+markers', name='% of Good Purpose', line=dict(color='red'), marker=dict(size=6, color='red')))

# Add bar for qEmployed
fig.add_trace(go.Bar(x=qEmployed_counts['Category'],y=qEmployed_counts['Count'],name='qEmployed',marker=dict(color='purple')))
emp_good_pct = x_train2.groupby('qEmployed')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=emp_good_pct.index,y=emp_good_pct.values,mode='lines+markers',name='% of Good Employed',line=dict(color='purple'),marker=dict(size=6, color='purple')))

# Add qSavings
fig.add_trace(go.Bar(x=qSavings_counts['Category'], y=qSavings_counts['Count'], name='qSavings', marker=dict(color='orange')))
sav_good_pct = x_train2.groupby('qSavings')['Good'].mean() * 100
fig.add_trace(go.Scatter(x=sav_good_pct.index, y=sav_good_pct.values, mode='lines+markers', name='% of Good Savings', line=dict(color='orange'), marker=dict(size=6, color='orange')))

# Update layout
fig.update_layout(
    title='Volume of Applicants and % of Goods by Category - subset 2 only',
    barmode='group',
    xaxis_title='Category',
    yaxis_title='Volume of Applicants',
    template='plotly_white'  # You can change the template for the look of the plot
)

# Show the plot
fig.show()

# Checking - keep as 4 buckets (clear difference in % goods in each bucket and logical difference between buckets)
# Coap - 1+2 vs. 3 (logical grouping of no guarantor vs. guarantor reflected in % of goods)
# Purpose - logical grouping: cars 1+2, small items 3,4,5, development 6,8,9, others 5,10
# Employed - logical groupings: 1,2,3,4+5 or 1,2+3,4+5
# Savings - logical groupings: 1,2,3,4+5 or 1,2, 3+4+5, or 1+2, 3+4+5


In [ ]:
# x2 Checking

# Keeping original categories
# IV = 0.75, but this is expected for this variable

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_checking = pd.crosstab(x_train2['qChecking'], x_train2['Good'], dropna=False).reset_index()
x2_checking.columns = ['Bin', 'Bads', 'Goods']
x2_checking = x2_checking[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_checking['Good%'] = x2_checking['Goods'] / x2_checking['Goods'].sum()
x2_checking['Bad%'] = x2_checking['Bads'] / x2_checking['Bads'].sum()

# Calculate WOE for each row
x2_checking['WOE'] = np.log(x2_checking['Good%'] / x2_checking['Bad%'])

# Calculate IV for each row
x2_checking['IV'] = (x2_checking['Good%'] - x2_checking['Bad%']) * x2_checking['WOE']

# Calculate total IV
total_IV_chk2 = x2_checking['IV'].sum()

# Print results
print(x2_checking)
print(f"Total IV for x2_checking: {total_IV_chk2}")

In [ ]:
# x2 Coapp

# 2 bins - very low IV
# 3 bins - very low IV

# Decision - don't use: very low IV

##################
# 2 bins
##################

bins_coapp1 = [0, 2, np.inf]
labels_coapp1 = ['No Guarantor', 'Guarantor']

# Bin the 'qCoapp' values
x_train2['qCoapp_bin'] = pd.cut(x_train2['qCoapp'], bins=bins_coapp1, labels=labels_coapp1, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_coap = pd.crosstab(x_train2['qCoapp_bin'], x_train2['Good'], dropna=False).reset_index()
x2_coap.columns = ['Bin', 'Bads', 'Goods']
x2_coap = x2_coap[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_coap['Good%'] = x2_coap['Goods'] / x2_coap['Goods'].sum()
x2_coap['Bad%'] = (x2_coap['Bads'] + ep) / x2_coap['Bads'].sum()

# Calculate WOE for each row
x2_coap['WOE'] = np.log(x2_coap['Good%'] / x2_coap['Bad%'])

# Calculate IV for each row
x2_coap['IV'] = (x2_coap['Good%'] - x2_coap['Bad%']) * x2_coap['WOE']

# Calculate total IV
total_IV_coap = x2_coap['IV'].sum()

# Print results
print(x2_coap)
print(f"Total IV for x2_coap: {total_IV_coap}")

##################
# 3 bins
##################

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_coap = pd.crosstab(x_train2['qCoapp'], x_train2['Good'], dropna=False).reset_index()
x2_coap.columns = ['Bin', 'Bads', 'Goods']
x2_coap = x2_coap[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_coap['Good%'] = x2_coap['Goods'] / x2_coap['Goods'].sum()
x2_coap['Bad%'] = (x2_coap['Bads'] + ep) / x2_coap['Bads'].sum()

# Calculate WOE for each row
x2_coap['WOE'] = np.log(x2_coap['Good%'] / x2_coap['Bad%'])

# Calculate IV for each row
x2_coap['IV'] = (x2_coap['Good%'] - x2_coap['Bad%']) * x2_coap['WOE']

# Calculate total IV
total_IV_coap = x2_coap['IV'].sum()

# Print results
print(x2_coap)
print(f"Total IV for x2_coap: {total_IV_coap}")


In [ ]:
# X2 Purpose
#Purpose - logical grouping: cars 0+1, small items 2,3,4, development 6,8,9, others 5,10

# Decision - Very low IV - don't include

purpose_mapping = {
    0: 'Cars',
    1: 'Cars',
    2: 'Small Items',
    3: 'Small Items',
    4: 'Small Items',
    5: 'Others',
    6: 'Development',
    8: 'Development',
    9: 'Development',
    10: 'Others'
}

# Bin the 'qPurpose' values
x_train2['qPurp_bin'] = x_train2['qPurpose'].map(purpose_mapping)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_purp = pd.crosstab(x_train2['qPurp_bin'], x_train2['Good'], dropna=False).reset_index()
x2_purp.columns = ['Bin', 'Bads', 'Goods']
x2_purp = x2_purp[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_purp['Good%'] = x2_purp['Goods'] / x2_purp['Goods'].sum()
x2_purp['Bad%'] = (x2_purp['Bads'] + ep) / x2_purp['Bads'].sum()

# Calculate WOE for each row
x2_purp['WOE'] = np.log(x2_purp['Good%'] / x2_purp['Bad%'])

# Calculate IV for each row
x2_purp['IV'] = (x2_purp['Good%'] - x2_purp['Bad%']) * x2_purp['WOE']

# Calculate total IV
total_IV_purp = x2_purp['IV'].sum()

# Print results
print(x2_purp)
print(f"Total IV for x2_purp: {total_IV_purp}")

In [ ]:
# x2

# Employed

# Decision - 3 bins - IV of 0.119 only slightly lower than 4 bins

##################
# Option 1: 1, 2, 3, 4+5
##################

# Define bins and labels for Option 1
bins_emp1 = [0, 1, 2, 3, np.inf]
labels_emp1 = ['1', '2', '3', '4+5']

# Bin the 'qEmployed' values
x_train2['qEmployed_bin1'] = pd.cut(x_train2['qEmployed'], bins=bins_emp1, labels=labels_emp1, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_emp1 = pd.crosstab(x_train2['qEmployed_bin1'], x_train2['Good'], dropna=False).reset_index()
x2_emp1.columns = ['Bin', 'Bads', 'Goods']
x2_emp1 = x2_emp1[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_emp1['Good%'] = x2_emp1['Goods'] / x2_emp1['Goods'].sum()
x2_emp1['Bad%'] = (x2_emp1['Bads'] + ep) / x2_emp1['Bads'].sum()

# Calculate WOE for each row
x2_emp1['WOE'] = np.log(x2_emp1['Good%'] / x2_emp1['Bad%'])

# Calculate IV for each row
x2_emp1['IV'] = (x2_emp1['Good%'] - x2_emp1['Bad%']) * x2_emp1['WOE']

# Calculate total IV
total_IV_emp1 = x2_emp1['IV'].sum()

# Print results
print(x2_emp1)
print(f"Total IV for x2_emp1: {total_IV_emp1}")


##################
# Option 2: 1, 2+3, 4+5
##################

# Define bins and labels for Option 2
bins_emp2 = [0, 1, 3, np.inf]
labels_emp2 = ['1', '2+3', '4+5']

# Bin the 'qEmployed' values
x_train2['qEmployed_bin2'] = pd.cut(x_train2['qEmployed'], bins=bins_emp2, labels=labels_emp2, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_emp2 = pd.crosstab(x_train2['qEmployed_bin2'], x_train2['Good'], dropna=False).reset_index()
x2_emp2.columns = ['Bin', 'Bads', 'Goods']
x2_emp2 = x2_emp2[['Bin', 'Goods', 'Bads']]  # Re-order columns

# Calculate G_i/G and B_i/B
x2_emp2['Good%'] = x2_emp2['Goods'] / x2_emp2['Goods'].sum()
x2_emp2['Bad%'] = (x2_emp2['Bads'] + ep) / x2_emp2['Bads'].sum()

# Calculate WOE for each row
x2_emp2['WOE'] = np.log(x2_emp2['Good%'] / x2_emp2['Bad%'])

# Calculate IV for each row
x2_emp2['IV'] = (x2_emp2['Good%'] - x2_emp2['Bad%']) * x2_emp2['WOE']

# Calculate total IV
total_IV_emp2 = x2_emp2['IV'].sum()

# Print results
print(x2_emp2)
print(f"Total IV for x2_emp2: {total_IV_emp2}")


In [ ]:
# x2
# Savings

# Decision - include with 3 bins - slightly higher IV of 0.2

##################
# Option 1: 1, 2, 3+4+5
##################

# Define bins and labels for Option 1
bins_sav1 = [0, 1, 2, np.inf]
labels_sav1 = ['1', '2', '3+4+5']

# Bin 'qSavings'
x_train2.loc[:, 'qSavings_bin1'] = pd.cut(x_train2['qSavings'], bins=bins_sav1, labels=labels_sav1, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_sav1 = pd.crosstab(x_train2['qSavings_bin1'], x_train2['Good'], dropna=False).reset_index()
x2_sav1.columns = ['Bin', 'Bads', 'Goods']
x2_sav1 = x2_sav1[['Bin', 'Goods', 'Bads']]

# Calculate G_i/G and B_i/B
x2_sav1['Good%'] = x2_sav1['Goods'] / x2_sav1['Goods'].sum()
x2_sav1['Bad%'] = (x2_sav1['Bads'] + ep) / x2_sav1['Bads'].sum()

# Calculate WOE for each row
x2_sav1['WOE'] = np.log(x2_sav1['Good%'] / x2_sav1['Bad%'])

# Calculate IV for each row
x2_sav1['IV'] = (x2_sav1['Good%'] - x2_sav1['Bad%']) * x2_sav1['WOE']

# Calculate total IV
total_IV_sav1 = x2_sav1['IV'].sum()

# Print results
print(x2_sav1)
print(f"Total IV for x2_sav1: {total_IV_sav1}")


##################
# Option 2: 1+2, 3+4+5
##################

# Define bins and labels for Option 2
bins_sav2 = [0, 2, np.inf]
labels_sav2 = ['1+2', '3+4+5']

# Bin 'qSavings'
x_train2.loc[:, 'qSavings_bin2'] = pd.cut(x_train2['qSavings'], bins=bins_sav2, labels=labels_sav2, right=True)

# Create a DataFrame with counts of 'Goods' and 'Bads' per bin
x2_sav2 = pd.crosstab(x_train2['qSavings_bin2'], x_train2['Good'], dropna=False).reset_index()
x2_sav2.columns = ['Bin', 'Bads', 'Goods']
x2_sav2 = x2_sav2[['Bin', 'Goods', 'Bads']]

# Calculate G_i/G and B_i/B
x2_sav2['Good%'] = x2_sav2['Goods'] / x2_sav2['Goods'].sum()
x2_sav2['Bad%'] = (x2_sav2['Bads'] + ep) / x2_sav2['Bads'].sum()

# Calculate WOE for each row
x2_sav2['WOE'] = np.log(x2_sav2['Good%'] / x2_sav2['Bad%'])

# Calculate IV for each row
x2_sav2['IV'] = (x2_sav2['Good%'] - x2_sav2['Bad%']) * x2_sav2['WOE']

# Calculate total IV
total_IV_sav2 = x2_sav2['IV'].sum()

# Print results
print(x2_sav2)
print(f"Total IV for x2_sav2: {total_IV_sav2}")


In [ ]:
#### FIGURE 1 ####

# Define features in order for x1
features_x1 = ['qCoapp', 'qChecking', 'nDuration', 'qProperty', 'nInstallp', 'nAge']

# Get Lasso coefficients for x1
lasso_x1 = [
    coef_df.loc[coef_df['feature'] == 'qCoapp_3', 'coef1'].values[0],
    coef_df.loc[coef_df['feature'] == 'qChecking_4', 'coef1'].values[0],
    coef_df.loc[coef_df['feature'] == 'nDuration', 'coef1'].values[0],
    coef_df.loc[coef_df['feature'] == 'qProperty_4', 'coef1'].values[0],
    coef_df.loc[coef_df['feature'] == 'nInstallp', 'coef1'].values[0],
    coef_df.loc[coef_df['feature'] == 'nAge', 'coef1'].values[0]
]

# Get IVs for x1
iv_x1 = [
    x1_coap['IV'].sum(),
    x1_checking['IV'].sum(),
    x1_dur_o2['IV'].sum(),
    x1_prop['IV'].sum(),
    x1_installp_o3['IV'].sum(),
    x1_age['IV'].sum()
]

# Define features in order for x2
features_x2 = ['qChecking', 'qSavings', 'nDuration', 'qEmployed', 'nAmount', 'qCoapp', 'qPurpose', 'nInstallp']

# Get Lasso coefficients for x2
lasso_x2 = [
    coef_df.loc[coef_df['feature'] == 'qChecking_4', 'coef2'].values[0],
    coef_df.loc[coef_df['feature'] == 'qSavings_5', 'coef2'].values[0],
    coef_df.loc[coef_df['feature'] == 'nDuration', 'coef2'].values[0],
    coef_df.loc[coef_df['feature'] == 'qEmployed_4', 'coef2'].values[0],
    coef_df.loc[coef_df['feature'] == 'nAmount', 'coef2'].values[0],
    coef_df.loc[coef_df['feature'] == 'qCoapp_3', 'coef2'].values[0],
    coef_df.loc[coef_df['feature'] == 'qPurpose_1', 'coef2'].values[0],
    coef_df.loc[coef_df['feature'] == 'nInstallp', 'coef2'].values[0]
]

# Get IVs for x2
iv_x2 = [
    x2_checking['IV'].sum(),
    x2_sav1['IV'].sum(),
    x2_dur_o2['IV'].sum(),
    x2_emp2['IV'].sum(),
    x2_amount_o2['IV'].sum(),
    x2_coap['IV'].sum(),
    x2_purp['IV'].sum(),
    x2_installp_o3['IV'].sum()
]

# Plot for x1
fig_x1 = go.Figure()
fig_x1.add_trace(go.Bar(x=features_x1, y=lasso_x1, name='Lasso Coefficients', marker_color='blue', text=[f"{v:.2f}" for v in lasso_x1], textposition='inside'))
fig_x1.add_trace(go.Bar(x=features_x1, y=iv_x1, name='IV (Information Value)', marker_color='orange', text=[f"{v:.2f}" for v in iv_x1], textposition='inside'))
fig_x1.update_layout(title='x1: Lasso Coefficients and IVs', xaxis=dict(title='Features'), yaxis=dict(title='Lasso Coefficients & IV'), barmode='group')

# Plot for x2
fig_x2 = go.Figure()
fig_x2.add_trace(go.Bar(x=features_x2, y=lasso_x2, name='Lasso Coefficients', marker_color='blue', text=[f"{v:.2f}" for v in lasso_x2], textposition='inside'))
fig_x2.add_trace(go.Bar(x=features_x2, y=iv_x2, name='IV (Information Value)', marker_color='orange', text=[f"{v:.2f}" for v in iv_x2], textposition='inside'))
fig_x2.update_layout(title='x2: Lasso Coefficients and IVs', xaxis=dict(title='Features'), yaxis=dict(title='Lasso Coefficients & IV'), barmode='group')

# Show the figures
fig_x1.show()
fig_x2.show()



In [ ]:
# Summary of binning decisions

# Plot 2 bar charts, one for df1, one for df2. For each binning option plot a stacked bar chart of IV value.

In [ ]:
#### FIGURE 2 ####

# Define labels and IV values for x1
features_x1_iv = ['qChecking', 'nDuration_o1', 'nDuration_o2', 'nDuration_o3', 'qProperty', 'nInstallp_o1', 'nInstallp_o2', 'nInstallp_o3']
iv_x1_values = [
    x1_checking['IV'].sum(),
    x1_dur_o1['IV'].sum(),
    x1_dur_o2['IV'].sum(),
    x1_dur_o3['IV'].sum(),
    x1_prop['IV'].sum(),
    x1_installp_o1['IV'].sum(),
    x1_installp_o2['IV'].sum(),
    x1_installp_o3['IV'].sum()
]

# Define labels and IV values for x2
features_x2_iv = ['qChecking', 'qSavings_o1', 'qSavings_o2', 'nDuration_o1', 'nDuration_o2', 'qEmployed_o1', 'qEmployed_o2']
iv_x2_values = [
    x2_checking['IV'].sum(),
    x2_sav1['IV'].sum(),
    x2_sav2['IV'].sum(),
    x2_dur_o1['IV'].sum(),
    x2_dur_o2['IV'].sum(),
    x2_emp1['IV'].sum(),
    x2_emp2['IV'].sum()
]

# Define colors for x1
colors_x1 = ['green' if feature in ['qChecking', 'nDuration_o2', 'qProperty', 'nInstallp_o3'] else 'red' if feature == 'nDuration_o3' else 'grey' for feature in features_x1_iv]

# Define colors for x2
colors_x2 = ['green' if feature in ['qChecking', 'qSavings_o1', 'nDuration_o2', 'qEmployed_o2'] else 'grey' for feature in features_x2_iv]

# Plot for x1
fig_iv_x1 = go.Figure()
fig_iv_x1.add_trace(go.Bar(x=features_x1_iv, y=iv_x1_values, name='IV (x1)', marker_color=colors_x1, text=[f"{v:.3f}" for v in iv_x1_values], textposition='inside'))

# Add annotation for nDuration_o3 on x1
fig_iv_x1.add_annotation(x='nDuration_o3', y=x1_dur_o3['IV'].sum(), text="IV too high, nDuration_o2 selected instead", showarrow=True, arrowhead=2, ax=40, ay=-40, font=dict(color="red"))

fig_iv_x1.update_layout(title='Subset 1: IV scores for all considered binning options for the 4 chosen variables', xaxis=dict(title='Features'), yaxis=dict(title='IV Values'), barmode='group')

# Plot for x2
fig_iv_x2 = go.Figure()
fig_iv_x2.add_trace(go.Bar(x=features_x2_iv, y=iv_x2_values, name='IV (x2)', marker_color=colors_x2, text=[f"{v:.3f}" for v in iv_x2_values], textposition='inside'))

# Add annotation for qEmployed_o2 on x2
fig_iv_x2.add_annotation(x='qEmployed_o2', y=x2_emp2['IV'].sum(), text="This option has 3 bins and<br>only slightly lower IV than o1 which has 4 bins", showarrow=True, arrowhead=2, ax=40, ay=-40, font=dict(color="green"))

fig_iv_x2.update_layout(title='Subset 2: IV scores for all considered binning options for the 4 chosen variables', xaxis=dict(title='Features'), yaxis=dict(title='IV Values'), barmode='group')

# Show the figures
fig_iv_x1.show()
fig_iv_x2.show()



In [ ]:
# Create x_train1_f, x_train2_f within only the 4 selected variables in their selected bins

#####
#x1
#####

# Recreate numerical bins for safety
x_train1_fs['nInstallp_bin'] = pd.cut(x_train1_fs['nInstallp'], bins=bins_o3, labels=labels_o3, right=True)
x_train1_fs['nDuration_bin'] = pd.cut(x_train1_fs['nDuration'], bins=bins_x1dur2, labels=labels_x1dur2, right=True)

# Create x_train1_f DataFrame
x_train1_f = pd.DataFrame({
    'qChecking': x_train1['qChecking'],
    'qProp': x_train1['qProp_bin'],
    'nInstallp': x_train1_fs['nInstallp_bin'],
    'nDuration': x_train1_fs['nDuration_bin']
})

# Ensure x_train1_f matches the indices of y_train1
x_train1_f = x_train1_f.loc[y_train1.index]

#####
#x2
#####

# Recreate numerical bins for safety
x_train2_fs['nDuration_bin'] = pd.cut(x_train2_fs['nDuration'], bins=bins_dur2, labels=labels_dur2, right=True)

# Create x_train2_f DataFrame
x_train2_f = pd.DataFrame({
    'qChecking': x_train2['qChecking'],
    'qEmployed': x_train2['qEmployed_bin2'],
    'qSavings': x_train2['qSavings_bin1'],
    'nDuration': x_train2_fs['nDuration_bin']
})

# Ensure x_train2_f matches the indices of y_train2
x_train2_f = x_train2_f.loc[y_train2.index]

print(f"x_train1_f shape: {x_train1_f.shape}")
print(f"x_train2_f shape: {x_train2_f.shape}")


In [ ]:
# Create test sets with appropriately binned chosen variables

#####
# x1 - Test
#####

# Unscale x_test1 using scaler1
x_test1_fs = pd.DataFrame(scaler1.inverse_transform(x_test1[numerical_cols]), columns=x_test1[numerical_cols].columns, index=x_test1.index)

# Create numerical bins
x_test1_fs['nInstallp_bin'] = pd.cut(x_test1_fs['nInstallp'], bins=bins_o3, labels=labels_o3, right=True)
x_test1_fs['nDuration_bin'] = pd.cut(x_test1_fs['nDuration'], bins=bins_x1dur2, labels=labels_x1dur2, right=True)

# Create qProperty bins
x_test1['qProp_bin'] = pd.cut(x_test1['qProperty'], bins=bins_prop1, labels=labels_prop1, right=True)

# Create x_test1_f DataFrame
x_test1_f = pd.DataFrame({
    'qChecking': x_test1['qChecking'],
    'qProp': x_test1['qProp_bin'],
    'nInstallp': x_test1_fs['nInstallp_bin'],
    'nDuration': x_test1_fs['nDuration_bin']
})

# Ensure x_test1_f matches the indices of y_test1
x_test1_f = x_test1_f.loc[y_test1.index]

#####
# x2 - Test
#####

# Unscale x_test2 using scaler2
x_test2_fs = pd.DataFrame(scaler2.inverse_transform(x_test2[numerical_cols]), columns=x_test2[numerical_cols].columns, index=x_test2.index)

# Create numerical bins
x_test2_fs['nDuration_bin'] = pd.cut(x_test2_fs['nDuration'], bins=bins_dur2, labels=labels_dur2, right=True)

# Create qEmployed_bin2 and qSavings_bin1
x_test2['qEmployed_bin2'] = pd.cut(x_test2['qEmployed'], bins=bins_emp2, labels=labels_emp2, right=True)
x_test2['qSavings_bin1'] = pd.cut(x_test2['qSavings'], bins=bins_sav1, labels=labels_sav1, right=True)

# Create x_test2_f DataFrame
x_test2_f = pd.DataFrame({
    'qChecking': x_test2['qChecking'],
    'qEmployed': x_test2['qEmployed_bin2'],
    'qSavings': x_test2['qSavings_bin1'],
    'nDuration': x_test2_fs['nDuration_bin']
})

# Ensure x_test2_f matches the indices of y_test2
x_test2_f = x_test2_f.loc[y_test2.index]

# Print shapes for confirmation
print(f"x_test1_f shape: {x_test1_f.shape}") # should be 72
print(f"x_test2_f shape: {x_test2_f.shape}") # should be 129


In [ ]:
# One hot encode x_train1_f, x_train2_f

x_train1_enc_f = pd.get_dummies(x_train1_f, columns=['qChecking','qProp','nInstallp','nDuration'], drop_first=True, dtype=int)
x_train2_enc_f = pd.get_dummies(x_train2_f, columns=['qChecking','qEmployed','qSavings','nDuration'], drop_first=True, dtype=int)

x_test1_enc_f = pd.get_dummies(x_test1_f, columns=['qChecking','qProp','nInstallp','nDuration'], drop_first=True, dtype=int)
x_test2_enc_f = pd.get_dummies(x_test2_f, columns=['qChecking','qEmployed','qSavings','nDuration'], drop_first=True, dtype=int)

# Create list of names of encoded categorical columns
categorical_cols_enc1 = [col for col in x_train1_enc_f.columns]
print(f"Categorical columns x_train1: {categorical_cols_enc1}")
categorical_cols_enc1_ts = [col for col in x_test1_enc_f.columns]
print(f"Categorical columns x_test1: {categorical_cols_enc1_ts}")
categorical_cols_enc2 = [col for col in x_train2_enc_f.columns]
print(f"Categorical columns x_train2: {categorical_cols_enc2}")
categorical_cols_enc2_ts = [col for col in x_test2_enc_f.columns]
print(f"Categorical columns x_test2: {categorical_cols_enc2_ts}")

# Section 4

In [ ]:
# Add a constant for statsmodels
xtn_1 = sm.add_constant(x_train1_enc_f)
xtn_2 = sm.add_constant(x_train2_enc_f)

# Define factor and offset - same for all models
factor = round(20 / np.log(2), 2)
offset = round(600 - factor * np.log(50), 2)

factor, offset

#### Fit all models

In [ ]:
# Subset 1 (<=12 months) - Linear Regression

lin_model1 = sm.OLS(y_train1, xtn_1)
lin_res1 = lin_model1.fit()
print(lin_res1.summary())
y_tn1_lin_pp = lin_res1.predict(xtn_1)
y_tn1_lin_pp = np.clip(y_tn1_lin_pp, 0, 1-ep) # clipping to prevent negative or infinite probabilty predictions which would result in NaN score

In [ ]:
# Subset 1 (<=12 months) - Logistic Regression

log_model1 = sm.Logit(y_train1, xtn_1)
log_res1 = log_model1.fit()
print(log_res1.summary())
y_tn1_log_pp = log_res1.predict(xtn_1)

In [ ]:
# Subset 2 (>12 months) - Linear Regression

lin_model2 = sm.OLS(y_train2, xtn_2)
lin_res2 = lin_model2.fit()
print(lin_res2.summary())
y_tn2_lin_pp = lin_res2.predict(xtn_2)
y_tn2_lin_pp = np.clip(y_tn2_lin_pp, 0, 1-ep) # clipping to prevent negative or infinite probabilty predictions which would result in NaN score

In [ ]:
# Subset 2 (>12 months) - Logistic Regression

log_model2 = sm.Logit(y_train2, xtn_2)
log_res2 = log_model2.fit()
print(log_res2.summary())
y_tn2_log_pp = log_res2.predict(xtn_2)

#### Calculate scores

In [ ]:
# Subset 1 (<=12 months) - Linear Regression

# calculate score
xtn_1['P1_lin1'] = y_tn1_lin_pp
xtn_1['P0_lin1'] = 1-y_tn1_lin_pp # this is the complement of P_1
xtn_1['Score_lin'] = np.ceil(np.log(xtn_1['P1_lin1']/xtn_1['P0_lin1']) * factor + offset)
xtn_1
xtn_1['Score_lin'].describe()

In [ ]:
# Subset 1 (<=12 months) - Logistic Regression

# calculate score
xtn_1['P1_log1'] = y_tn1_log_pp
xtn_1['P0_log1'] = 1-y_tn1_log_pp # this is the complement of P_1
xtn_1['Score_log'] = np.ceil(np.log(xtn_1['P1_log1']/xtn_1['P0_log1']) * factor + offset)
xtn_1
xtn_1['Score_log'].describe()

In [ ]:
xtn_1

In [ ]:
# Subset 2 (>12 months) - Linear Regression

# calculate score
xtn_2['P1_lin2'] = y_tn2_lin_pp
xtn_2['P0_lin2'] = 1-y_tn2_lin_pp # this is the complement of P_1
xtn_2['Score_lin'] = np.ceil(np.log(xtn_2['P1_lin2']/xtn_2['P0_lin2']) * factor + offset)
xtn_2
xtn_2['Score_lin'].describe()

In [ ]:
# Subset 2 (>12 months) - Logistic Regression

# calculate score
xtn_2['P1_log2'] = y_tn2_log_pp
xtn_2['P0_log2'] = 1-y_tn2_log_pp # this is the complement of P_1
xtn_2['Score_log'] = np.ceil(np.log(xtn_2['P1_log2']/xtn_2['P0_log2']) * factor + offset)
xtn_2
xtn_2['Score_log'].describe()

In [ ]:
xtn_2

*Coefficient tables for appendix*

In [ ]:
# lin1

# Extract regression coefficients into a df
param_lin1 = lin_res1.params
coef_lin1 = pd.DataFrame({'Var': param_lin1.index, 'Coef': param_lin1.values})
coef_lin1

In [ ]:
# log1

# Extract regression coefficients into a df
param_log1 = log_res1.params
coef_log1 = pd.DataFrame({'Var': param_log1.index, 'Coef': param_log1.values})

# Add missing bins for each variable
add_rows1 = pd.DataFrame({
    'Var': [
        'checking_ref',
        'property_ref',
        'installp_ref',
        'duration_ref'
    ],
    'Coef': [0,0,0,0]
})

# Specify insert position
insert_after = {
    'qChecking_4': 'checking_ref',
    'qProp_Unk_Na': 'property_ref',
    'nInstallp_4+ Installp': 'installp_ref',
    'nDuration_12': 'duration_ref'
}

# Results df
coef_log11 = coef_log1.copy()

# Insert
for position, new_row in insert_after.items():
  idx = coef_log11.index[coef_log11['Var'] == position][0]+1
  top = coef_log11.iloc[:idx, :]
  bottom = coef_log11.iloc[idx:,:]
  new_row = pd.DataFrame([[new_row, 0]], columns=coef_log11.columns)
  coef_log11 = pd.concat([top, new_row, bottom], ignore_index=True)

coef_log11

In [ ]:
# lin2

# Extract regression coefficients into a df
param_lin2 = lin_res2.params
coef_lin2 = pd.DataFrame({'Var': param_lin2.index, 'Coef': param_lin2.values})
coef_lin2

In [ ]:
# log2

# Extract regression coefficients into a df
param_log2 = log_res2.params
coef_log2 = pd.DataFrame({'Var': param_log2.index, 'Coef': param_log2.values})

# Add missing bins for each variable
add_rows2 = pd.DataFrame({
    'Var': [
        'checking_ref',
        'employed_ref',
        'savings_ref',
        'duration_ref'
    ],
    'Coef': [0, 0, 0, 0]
})

# Specify insert position
insert_after = {
    'qChecking_4': 'checking_ref',
    'qEmployed_4+5': 'employed_ref',
    'qSavings_3+4+5': 'savings_ref',
    'nDuration_48+': 'duration_ref'
}

# Results df
coef_log22 = coef_log2.copy()

# Insert
for position, new_row in insert_after.items():
    idx = coef_log22.index[coef_log22['Var'] == position][0] + 1
    top = coef_log22.iloc[:idx, :]
    bottom = coef_log22.iloc[idx:, :]
    new_row_df = pd.DataFrame([[new_row, 0]], columns=coef_log22.columns)
    coef_log22 = pd.concat([top, new_row_df, bottom], ignore_index=True)

coef_log22

# Section 5

Validation set:
* RoC curves
* Sensitivity and Specifity calculations
* Gini coefficient
* KS values

In [ ]:
# Predict the test set
# Add constant
xtt_1 = sm.add_constant(x_test1_enc_f)
xtt_2 = sm.add_constant(x_test2_enc_f)

# Predictions
y_tt1_lin_pp = lin_res1.predict(xtt_1)
y_tt1_lin_pp = np.clip(y_tt1_lin_pp, 0, 1-ep)# clip
y_tt1_log_pp = log_res1.predict(xtt_1)
y_tt2_lin_pp = lin_res2.predict(xtt_2)
y_tt2_lin_pp = np.clip(y_tt2_lin_pp, 0, 1-ep)# clip
y_tt2_log_pp = log_res2.predict(xtt_2)


In [ ]:
# ROC curves

# Define models and test sets
models_test1 = {
    'y_tt1_lin_pp': y_tt1_lin_pp,
    'y_tt1_log_pp': y_tt1_log_pp
}

models_test2 = {
    'y_tt2_lin_pp': y_tt2_lin_pp,
    'y_tt2_log_pp': y_tt2_log_pp
}

# Plot ROC curves for y_test1
plt.figure(figsize=(8, 6))
for model_name, y_pred in models_test1.items():
    # Compute ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test1, y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.2f})')

# Plot diagonal reference line
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for subset 1 (Duration <=12 months)')
plt.legend(loc="lower right")
plt.show()

# Plot ROC curves for y_test2
plt.figure(figsize=(8, 6))
for model_name, y_pred in models_test2.items():
    # Compute ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_test2, y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot
    plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.2f})')

# Plot diagonal reference line
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for subset 2 (Duration >12 months)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Specificity and Sensitivity

# Specificity = TN/(TN/FP) = True Negative Rate
# Sensitivity = TP/(TP/FN) = True Positive Rate = Recall

# Define model predictions and test sets
predictions = {
    'y_tt1_lin_pp': y_tt1_lin_pp,
    'y_tt1_log_pp': y_tt1_log_pp,
    'y_tt2_lin_pp': y_tt2_lin_pp,
    'y_tt2_log_pp': y_tt2_log_pp
}

test_sets = {
    'y_tt1_lin_pp': y_test1,
    'y_tt1_log_pp': y_test1,
    'y_tt2_lin_pp': y_test2,
    'y_tt2_log_pp': y_test2
}

# Loop through predictions
for name, y_pred in predictions.items():
    # Generate class predictions with 0.5 threshold
    y_class = (y_pred > 0.5).astype(int)

    # Get the corresponding test set
    y_test = test_sets[name]

    # Generate the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_class)
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate Specificity and Sensitivity
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)

    # Plot confusion matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Bad', 'Good'], yticklabels=['Bad', 'Good'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix for {name}')
    plt.show()

    # Print results
    print(f"Specificity (True Negative Rate) for {name}: {specificity:.2f}")
    print(f"Sensitivity (True Positive Rate/Recall) for {name}: {sensitivity:.2f}\n")

In [ ]:
# Gini coefficient

# G = 2AUROC-1

# Calculate AUROC for each model
# Subset 1 - linear
fpr_lin1, tpr_lin1, thresholds_lin1 = roc_curve(y_test1, y_tt1_lin_pp)
roc_auc_lin1 = auc(fpr_lin1, tpr_lin1)
# Subset 1 - logistic
fpr_log1, tpr_log1, thresholds_log1 = roc_curve(y_test1, y_tt1_log_pp)
roc_auc_log1 = auc(fpr_log1, tpr_log1)

# Subset 2 - linear
fpr_lin2, tpr_lin2, thresholds_lin2 = roc_curve(y_test2, y_tt2_lin_pp)
roc_auc_lin2 = auc(fpr_lin2, tpr_lin2)
# Subset 2 - logistic
fpr_log2, tpr_log2, thresholds_log2 = roc_curve(y_test2, y_tt2_log_pp)
roc_auc_log2 = auc(fpr_log2, tpr_log2)

# Calculate Gini Coefficients for each model
gini_lin1 = 2 * roc_auc_lin1 - 1
gini_log1 = 2 * roc_auc_log1 - 1

gini_lin2 = 2 * roc_auc_lin2 - 1
gini_log2 = 2 * roc_auc_log2 - 1


# Print Gini Coefficients for each model
print(f"Gini coefficient for subset 1 linear regression: {gini_lin1:.2f}")
print(f"Gini coefficient for subset 1 logistic regression: {gini_log1:.2f}\n")
print(f"Gini coefficient for subset 2 linear regression: {gini_lin2:.2f}")
print(f"Gini coefficient for subset 2 logistic regression: {gini_log2:.2f}\n")

In [ ]:
# Kolmogorov-Smirnov Value

# maximum absolute difference in F(s|B) and F(s|G)

# Calculate scores for each sample in the test set

# Subset 1 (<=12 months) - LinearRegression
# calculate score
xtt_1['P1_lin1'] = y_tt1_lin_pp
xtt_1['P0_lin1'] = 1-y_tt1_lin_pp
xtt_1['Score_lin'] = np.ceil(np.log(xtt_1['P1_lin1']/xtt_1['P0_lin1']) * factor + offset)
xtt_1
#print(xtt_1['Score_lin'].describe())

# Subset 1 (<=12 months) - Logistic Regression
# calculate score
xtt_1['P1_log1'] = y_tt1_log_pp
xtt_1['P0_log1'] = 1-y_tt1_log_pp
xtt_1['Score_log'] = np.ceil(np.log(xtt_1['P1_log1']/xtt_1['P0_log1']) * factor + offset)
xtt_1
#print(xtt_1['Score_log'].describe())


# Subset 2 (>12 months) - LinearRegression
# calculate score
xtt_2['P1_lin2'] = y_tt2_lin_pp
xtt_2['P0_lin2'] = 1-y_tt2_lin_pp
xtt_2['Score_lin'] = np.ceil(np.log(xtt_2['P1_lin2']/xtt_2['P0_lin2']) * factor + offset)
xtt_2
#print(xtt_2['Score_lin'].describe())

# Subset 2 (>12 months) - Logistic Regression
# calculate score
xtt_2['P1_log2'] = y_tt2_log_pp
xtt_2['P0_log2'] = 1-y_tt2_log_pp
xtt_2['Score_log'] = np.ceil(np.log(xtt_2['P1_log2']/xtt_2['P0_log2']) * factor + offset)
xtt_2
#print(xtt_2['Score_log'].describe())

In [ ]:
# df for Kolmogorov-Smirnov Value for Subset 1 (<=12 months) - LinearRegression
ks_lin1 = pd.DataFrame({
    'Score': xtt_1['Score_lin'],
    'Actual': y_test1
}).sort_values(by='Score', ascending=True)

# cumulative FN
ks_lin1['FN'] = ks_lin1['Actual'].cumsum()

# cumulative TN
ks_lin1['TN'] = ks_lin1['Actual'].eq(0).cumsum()

# Add cumulative FP
ks_lin1['FP'] = ks_lin1['TN'].iloc[-1] - ks_lin1['TN']

# Add cumulative TP
ks_lin1['TP'] = ks_lin1['FN'].iloc[-1] - ks_lin1['FN']

# F(s|G)
ks_lin1['F(s|G)'] = ks_lin1['FN'] / ks_lin1['FN'].iloc[-1]

# F(s|B)
ks_lin1['F(s|B)'] = ks_lin1['TN'] / ks_lin1['TN'].iloc[-1]

# Kolmogorov-Smirnov Value
ks_lin1['KS_Stat'] = abs(ks_lin1['F(s|G)'] - ks_lin1['F(s|B)'])

print(ks_lin1['KS_Stat'].max())
# ks_lin1

In [ ]:
# df for Kolmogorov-Smirnov Value for Subset 1 (<=12 months) - LogisticRegression
ks_log1 = pd.DataFrame({
    'Score': xtt_1['Score_log'],
    'Actual': y_test1
}).sort_values(by='Score', ascending=True)

# cumulative FN
ks_log1['FN'] = ks_log1['Actual'].cumsum()

# cumulative TN
ks_log1['TN'] = ks_log1['Actual'].eq(0).cumsum()

# Add cumulative FP
ks_log1['FP'] = ks_log1['TN'].iloc[-1] - ks_log1['TN']

# Add cumulative TP
ks_log1['TP'] = ks_log1['FN'].iloc[-1] - ks_log1['FN']

# F(s|G)
ks_log1['F(s|G)'] = ks_log1['FN'] / ks_log1['FN'].iloc[-1]

# F(s|B)
ks_log1['F(s|B)'] = ks_log1['TN'] / ks_log1['TN'].iloc[-1]

# Kolmogorov-Smirnov Value
ks_log1['KS_Stat'] = abs(ks_log1['F(s|G)'] - ks_log1['F(s|B)'])

print(ks_log1['KS_Stat'].max())
# ks_log1

In [ ]:
# df for Kolmogorov-Smirnov Value for Subset 2 (>12 months) - LinearRegression
ks_lin2 = pd.DataFrame({
    'Score': xtt_2['Score_lin'],
    'Actual': y_test2
}).sort_values(by='Score', ascending=True)

# cumulative FN
ks_lin2['FN'] = ks_lin2['Actual'].cumsum()

# cumulative TN
ks_lin2['TN'] = ks_lin2['Actual'].eq(0).cumsum()

# Add cumulative FP
ks_lin2['FP'] = ks_lin2['TN'].iloc[-1] - ks_lin2['TN']

# Add cumulative TP
ks_lin2['TP'] = ks_lin2['FN'].iloc[-1] - ks_lin2['FN']

# F(s|G)
ks_lin2['F(s|G)'] = ks_lin2['FN'] / ks_lin2['FN'].iloc[-1]

# F(s|B)
ks_lin2['F(s|B)'] = ks_lin2['TN'] / ks_lin2['TN'].iloc[-1]

# Kolmogorov-Smirnov Value
ks_lin2['KS_Stat'] = abs(ks_lin2['F(s|G)'] - ks_lin2['F(s|B)'])

print(ks_lin2['KS_Stat'].max())
# ks_lin2

In [ ]:
# Create DataFrame for Score_log - xtt_2
ks_log2 = pd.DataFrame({
    'Score': xtt_2['Score_log'],
    'Actual': y_test2
}).sort_values(by='Score', ascending=True)

# cumulative FN
ks_log2['FN'] = ks_log2['Actual'].cumsum()

# cumulative TN
ks_log2['TN'] = ks_log2['Actual'].eq(0).cumsum()

# Add cumulative FP
ks_log2['FP'] = ks_log2['TN'].iloc[-1] - ks_log2['TN']

# Add cumulative TP
ks_log2['TP'] = ks_log2['FN'].iloc[-1] - ks_log2['FN']

# F(s|G)
ks_log2['F(s|G)'] = ks_log2['FN'] / ks_log2['FN'].iloc[-1]

# F(s|B)
ks_log2['F(s|B)'] = ks_log2['TN'] / ks_log2['TN'].iloc[-1]

# Kolmogorov-Smirnov Value
ks_log2['KS_Stat'] = abs(ks_log2['F(s|G)'] - ks_log2['F(s|B)'])

print(ks_log2['KS_Stat'].max())
# ks_log2

In [ ]:
# Print KS results for each model

print(f"Kolmogrov-Smirnov statistic for subset 1 linear regression: {ks_lin1['KS_Stat'].max():.3f}")
print(f"Kolmogrov-Smirnov statistic for subset 1 logistic regression: {ks_log1['KS_Stat'].max():.3f}\n")
print(f"Kolmogrov-Smirnov statistic for subset 2 linear regression: {ks_lin2['KS_Stat'].max():.3f}")
print(f"Kolmogrov-Smirnov statistic for subset 2 logistic regression: {ks_log2['KS_Stat'].max():.3f}\n")

In [ ]:
# Plot KS curves for each model

# List of ks_ dataframes and their corresponding titles
ks_dfs = [ks_lin1, ks_log1, ks_lin2, ks_log2,]
titles = ['KS - <=12 months - Linear', 'KS <=12 months - Logistic', 'KS >12 months - Linear', 'KS >12 months - Logistic']

# Create a 2x2 subplot
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

# Loop over the ks_dfs and plot
for i, (ks_df, title) in enumerate(zip(ks_dfs, titles)):
    row = i // 2
    col = i % 2

    # Plot the F(s|G) and F(s|B) curves against score
    axs[row, col].plot(ks_df['Score'], ks_df['F(s|G)'], label='F(s|G)', color='blue')
    axs[row, col].plot(ks_df['Score'], ks_df['F(s|B)'], label='F(s|B)', color='orange')

    # Set labels and title
    axs[row, col].set_xlabel('Score')
    axs[row, col].set_ylabel('Cumulative Probability')
    axs[row, col].set_title(f'{title} - KS = {ks_df["KS_Stat"].max():.3f}')
    axs[row, col].legend()

# Adjust layout for better spacing
plt.tight_layout()
plt.show()